In [ ]:
import pandas as pd
import glob
import re
import numpy as np
import math
import sklearn
from sklearn.utils import shuffle 
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, \
accuracy_score, roc_curve, precision_recall_curve
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#Random feature importance to get feature importance threshold
def random_feature_importance(data):
    df_train = data[data.chr != 'chr1']
    df_test = data[data.chr == 'chr1']
    X_train = df_train.drop(columns = ['is_asb', 'chr'])
    X_val = df_test.drop(columns = ['is_asb', 'chr'])
    y_train = np.array(df_train.is_asb)
    y_test = np.array(df_test.is_asb)
    for column in X_train.columns:
        X_train.loc[:, column] = shuffle(np.asarray(X_train[[column]]), random_state=1)
    rndclf = RandomForestClassifier(random_state=1, n_estimators=500, n_jobs=36)
    rndclf.fit(X_train, y_train)
    mean_rnd_importance = np.mean(rndclf.feature_importances_)
    max_rnd_importance = np.max(rndclf.feature_importances_)
    return max_rnd_importance

In [ ]:
#Preparing for building ROC and PR curves
def curves(y_test, rfpred, val_chr):
    fpr, tpr, thresholds = roc_curve(y_test, rfpred[:, 1], pos_label=1)
    pr, rec, thresh_prauc = precision_recall_curve(y_test, rfpred[:, 1], pos_label=1)
    df_temp1 = pd.DataFrame(data={'val_chr': val_chr, 'fpr': fpr, 'tpr': tpr, 
                                    'thresholds_roc': thresholds})
    df_temp2 = pd.DataFrame(data={'val_chr': val_chr, 'precision':pr[:-1], 'recall':rec[:-1], 
                                    'thresholds_prauc': thresh_prauc})
    return df_temp1, df_temp2

In [ ]:
#Cross validation for individual model(one chromosome - test, all others - train)
def cross_validation_random_forest(data, rforest_name, roc_name, pr_name):
    df_roc = pd.DataFrame()
    df_pr = pd.DataFrame()
    rforest_file = open(rforest_name, 'w')
    header = '\t'.join(['val_chr', 'roc_auc', 'f1', 'accuracy', 'pr_auc', 'important_features', 'feature_scores'])
    rforest_file.write(header + '\n')
    max_rnd_importance = random_feature_importance(data)
    for i in range(1, 23):
        val_chr = 'chr' + str(i)
        df_train = data[data.chr != val_chr]
        df_test = data[data.chr == val_chr]
        if (len(df_test) == 0):
            continue
        X_train = df_train.drop(columns = ['is_asb', 'chr'])
        X_val = df_test.drop(columns = ['is_asb','chr'])
        y_train = np.array(df_train.is_asb)
        y_test = np.array(df_test.is_asb)
        if len(np.unique(y_test)) == 1:
            continue
        clf = RandomForestClassifier(n_estimators=500, random_state=1, n_jobs=36)
        clf.fit(X_train, y_train)
        rfpred = clf.predict_proba(X_val)
        pred_classes = clf.predict(X_val)
        important_features = X_train.columns[np.where(clf.feature_importances_ > max_rnd_importance)]
        important_features_scores = clf.feature_importances_[np.where(clf.feature_importances_ > max_rnd_importance)]
        roc_auc = roc_auc_score(y_test, rfpred[:, 1])
        f1 = f1_score(y_test, pred_classes)
        accuracy = accuracy_score(y_test, pred_classes)
        prscore = average_precision_score(y_test, rfpred[:, 1])
        string_scores = [str(s) for s in [roc_auc, f1, accuracy, prscore]]
        string_feature_scores = [str(s) for s in important_features_scores]
        rforest_file.write(val_chr + '\t' + '\t'.join(string_scores) + '\t' + ';'.join(important_features) + '\t' + ';'.join(string_feature_scores) + '\n')
        df_temp1, df_temp2 = curves(y_test, rfpred, val_chr)
        df_roc = df_roc.append(df_temp1)
        df_pr = df_pr.append(df_temp2)
    rforest_file.close()
    df_roc.to_csv(roc_name, sep='\t', index=False)
    df_pr.to_csv(pr_name, sep='\t', index=False)
    return 'Done'

In [ ]:
#Individual RnadomForest
def individual_RF(data_list, data_type='TFs'):
    for o in data_list:
        infilename = data_type + '/' + o + '.tsv'
        data = pd.read_csv(infilename, sep='\t')
        data.drop(columns = ['ID', 'name', 'pos', 'ref', 'alt'], inplace=True)
        #outfiles names
        filename = data_type + '_result/' + o + '_random_forest.tsv'
        roc_name = data_type + '_result/' + o + '_roc_curve.tsv'
        pr_name = data_type + '_result/' + o + '_pr_curve.tsv'
        print('Working with ' + o)
        cross_validation_random_forest(data, filename, roc_name, pr_name)

## Running individual models

In [ ]:
#Individual RandomForest fot top ten trancription factors
individual_RF(['CTCF', 'FOXA1', 'ANDR', 'ESR1', 'FOXK2', 'SPI1', 'STAT1', 'IKZF1', 'RAD21', 'CEBPB'])

In [ ]:
#Individual RandomForest fot top ten cell types
individual_RF(['K562__myelogenous_leukemia_', 'GM12878__female_B-cells_', 
               'HEK293__embryonic_kidney_', 'MCF7__Invasive_ductal_breast_carcinoma_', 
               'HepG2__hepatoblastoma_', 'foreskin_keratinocyte','A549__lung_carcinoma_', 
               'LNCaP__prostate_carcinoma_', 'VCaP__prostate_carcinoma_', 'CD14+_monocytes'], data_type='CLs')

## Running general models

In [ ]:
#Cross validation for global model
def global_RF(data, name_list, data_list, data_type='TFs'):
    for i in range(1, 23):
        val_chr = 'chr' + str(i)
        df_train = data[data.chr != val_chr]
        X_train = df_train.drop(columns = ['is_asb', 'chr'])
        y_train = np.array(df_train.is_asb)
        clf = RandomForestClassifier(n_estimators=500, random_state=1, n_jobs=36)
        clf.fit(X_train, y_train)
        for o_name, o_data in zip(name_list, data_list):
            df_test = o_data[o_data.chr == val_chr]
            y_test = np.array(df_test.is_asb)
            if len(np.unique(y_test)) < 2:
                continue
            X_val = df_test.drop(columns = ['is_asb','chr'])
            rfpred = clf.predict_proba(X_val)
            roc_auc = roc_auc_score(y_test, rfpred[:, 1])
            pr_score = average_precision_score(y_test, rfpred[:, 1])
            filename = data_type + '_result/' + o_name + '_combined.tsv'
            with open(filename, 'a') as f:
                f.write(val_chr + '\t' + str(roc_auc) + '\t' + str(pr_score) + '\n')
            df_roc, df_pr = curves(y_test, rfpred, val_chr)
            filename_roc = data_type + '_result/' + o_name + '_combined_roc.tsv'
            filename_pr = data_type + '_result/' + o_name + '_combined_pr.tsv'
            df_roc.to_csv(filename_roc, sep='\t', index=False, mode='a', header=None)
            df_pr.to_csv(filename_pr, sep='\t', index=False, mode='a', header=None)
        print(val_chr + ' done')
    
    

In [ ]:
data = pd.read_csv('deepsea_data_TF_220620.tsv', sep='\t')
data.drop(columns = ['name', 'ref', 'alt', 'pos'], inplace=True)

In [ ]:
tf_list = ['CTCF', 'FOXA1', 'ANDR', 'ESR1', 'FOXK2', 'SPI1', 'STAT1', 'IKZF1', 'RAD21', 'CEBPB']
tf_data_list = [pd.read_csv('TFs/' + tf + '.tsv', sep='\t') for tf in tf_list]
tf_data_list = [df.drop(columns = ['ID', 'name', 'pos', 'ref', 'alt'], inplace=True) for df in tf_data_list]

In [ ]:
global_RF(data, tf_list, tf_data_list, data_type='TFs')

In [ ]:
data = pd.read_csv('deepsea_data_CL_220620.tsv', sep='\t')
data.drop(columns = ['name', 'ref', 'alt', 'pos'], inplace=True)

In [ ]:
cl_list = ['K562__myelogenous_leukemia_', 'GM12878__female_B-cells_', 
               'HEK293__embryonic_kidney_', 'MCF7__Invasive_ductal_breast_carcinoma_', 
               'HepG2__hepatoblastoma_', 'foreskin_keratinocyte','A549__lung_carcinoma_', 
               'LNCaP__prostate_carcinoma_', 'VCaP__prostate_carcinoma_', 'CD14+_monocytes']
cl_data_list = [pd.read_csv('CLs/' + cl + '.tsv', sep='\t') for cl in cl_list]
cl_data_list = [df.drop(columns = ['ID', 'name', 'pos', 'ref', 'alt'], inplace=True) for df in cl_data_list]

In [ ]:
global_RF(data, cl_list, cl_data_list, data_type='CLs')